In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler, SequentialSampler
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)


In [2]:
import os 
import glob
import pandas as pd
from torch.utils.data import Dataset
import numpy as np
import torch
from glob import glob

class TimeSeriesDataset(Dataset):
    def __init__(self, folder_nolayoff, folder_layoff, src_col):
        
        # csvs_layoffs = glob.glob(folder_layoff + "/*.csv")
        # csvs_nolayoffs = glob.glob(folder_nolayoff + "/*.csv")
        X_train = []
        Y_train = []
        all_csv_files = []
        EXT = "*.csv"  # Define the variable EXT
        for path, subdir, files in os.walk(folder_layoff):
            
            for file in glob(os.path.join(path, EXT)):
                # print(file)
                df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
                ## If length of dataset is over 90, then cut it to 90
                if len(df) > 90:
                    print("OVER LENGTH", file)
                    df = df[-90:]
                ## If length of dataset is less than 90, then skip it
                if len(df) < 90:
                    continue
                X_train.append(df[src_col].values)
                Y_train.append(float(1))
                # all_csv_files.append(file)

        for path, subdir, files in os.walk(folder_nolayoff):
            for file in glob(os.path.join(path, EXT)):
                df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
                ## If length of dataset is over 90, then cut it to 90
                if len(df) > 90:
                    print("OVER LENGTH", file)
                    df = df[-90:]

                ## If length of dataset is less than 90, then skip it
                if len(df) < 90:
                    continue
                X_train.append(df[src_col].values)
                Y_train.append(float(0))
                # all_csv_files.append(file)





        # for directory in csvs_layoffs:
        #     for csv in os.listdir(directory):
        #         if csv.endswith(".csv"):
        #             df = pd.read_csv(csv, index_col=0, parse_dates=True).sort_index()
        #             X_train.append(df[src_col].values)
        #             Y_train.append(1)
        #     # df = pd.read_csv(csv, index_col=0, parse_dates=True).sort_index()
        #     # X_train.append(df[src_col].values)
        #     # Y_train.append(1)
        # for directory in csvs_nolayoffs:
        #     for csv in os.listdir(directory):
        #         if csv.endswith(".csv"):
        #             df = pd.read_csv(csv, index_col=0, parse_dates=True).sort_index()
        #             X_train.append(df[src_col].values)
        #             Y_train.append(0)
            
        X_train = torch.from_numpy(np.array(X_train))
        Y_train = torch.from_numpy(np.array(Y_train))
        # if len(X_train.shape) < 3:
        #     X_train = X_train.unsqueeze(2)

        # if X_train.shape.index(min(X_train.shape[1], X_train.shape[2])) != 1:  # make sure the Channels in second dim
        #     X_train = X_train.permute(0, 2, 1)

        self.X_train = X_train
        self.Y_train = Y_train
        self.num_channels = X_train.shape[1]
        self.len = X_train.shape[0]

    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        return self.X_train[idx].float(), self.Y_train[idx]



In [3]:
df = pd.read_csv("stocks_layoffs/Energy/ENPH0.csv")

In [4]:
# avg = list(df['Open'].rolling(window=90).mean())[-1]
# df['Normalized_Price_SMA90'] = df['Open'] / avg

In [5]:
# df["Normalized_Price_SMA90"]

In [6]:
dataset_sma90 = TimeSeriesDataset("stocks_no_layoffs", "stocks_layoffs", "Normalized_Price_SMA90")
dataset_oipa = TimeSeriesDataset("stocks_no_layoffs", "stocks_layoffs", "open_inproportion_to_average")
dataset_on = TimeSeriesDataset("stocks_no_layoffs", "stocks_layoffs", "open_normalized")
batch_size = 1
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(dataset_sma90)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SequentialSampler(val_indices)

train_loader_sma90 = torch.utils.data.DataLoader(dataset_sma90, batch_size=batch_size, 
                                           sampler=train_sampler)
train_loader_oipa = torch.utils.data.DataLoader(dataset_oipa, batch_size=batch_size, 
                                           sampler=train_sampler)
train_loader_on = torch.utils.data.DataLoader(dataset_on, batch_size=batch_size, 
                                           sampler=train_sampler)

validation_loader_sma90 = torch.utils.data.DataLoader(dataset_sma90, batch_size=batch_size, sampler=valid_sampler)
validation_loader_oipa = torch.utils.data.DataLoader(dataset_oipa, batch_size=batch_size, sampler=valid_sampler)
validation_loader_on = torch.utils.data.DataLoader(dataset_on, batch_size=batch_size,   sampler=valid_sampler)

In [7]:
from torch.utils.data import DataLoader

In [8]:
train_loader_sma90.dataset.Y_train

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [9]:
# tdloader = TimeSeriesDataset("stocks_no_layoffs", "stocks_layoffs", "open_in_proportion_to_average")

In [10]:
# layoff_data_loader = DataLoader(dataset=tdloader, batch_size=1, 
#                                 shuffle=True, drop_last=False, num_workers=0)

In [11]:
# layoff_data_loader.dataset.X_train.shape

In [12]:
# ## Iterate through dataloader and print y labels
# for i, (x, y) in enumerate(validation_loader_sma90):
#     print(y)
#     # print(x)
    # break

In [13]:
# ## Iterate through dataloader and print y labels
# for i, (x, y) in enumerate(train_loader_oipa):
#     print(y)

In [19]:
train_loader_oipa.dataset.Y_train

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [14]:
# Define the model architecture
class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(90, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Create an instance of the model
model_sma_90d = BinaryClassifier()
model_open_inproportion_to_average = BinaryClassifier()
model_open_normalized = BinaryClassifier()
# Define the loss function and optimizer
criterion_sma90 = nn.BCELoss()
criterion_oipa = nn.BCELoss()
criterion_on = nn.BCELoss()
optimizer_sma90 = optim.Adam(model_sma_90d.parameters())
optimizer_oipa = optim.Adam(model_open_inproportion_to_average.parameters())
optimizer_on = optim.Adam(model_open_normalized.parameters())

# Training loop
num_epochs = 50
batch_size = 32

for epoch in range(num_epochs):
    ## Run through dataloader batches
    



    for i, (X_train, y_train) in enumerate(train_loader_sma90):
    
        # Forward pass
        # print(X_train)
        outputs_sma_90d = model_sma_90d(X_train)
        y_train = y_train.unsqueeze(1)
        loss_sma90 = criterion_sma90(outputs_sma_90d.float(), y_train.float())
        optimizer_sma90.zero_grad()
        loss_sma90.backward()
        optimizer_sma90.step()
    for i, (X_train, y_train) in enumerate(train_loader_oipa):
        # Forward pass
        outputs_open_inproportion_to_average = model_open_inproportion_to_average(X_train)
        y_train = y_train.unsqueeze(1)
        loss_oipa = criterion_oipa(outputs_open_inproportion_to_average.float(), y_train.float())
        optimizer_oipa.zero_grad()
        loss_oipa.backward()
        optimizer_oipa.step()
    for i, (X_train, y_train) in enumerate(train_loader_on):
        # Forward pass
        outputs_open_normalized = model_open_normalized(X_train)
        y_train = y_train.unsqueeze(1)
        loss_on = criterion_on(outputs_open_normalized.float(), y_train.float())
        # Backward pass and optimization
        optimizer_on.zero_grad()
        loss_on.backward()
        optimizer_on.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss_sma90: {loss_sma90.item():.4f}, Loss_oipa: {loss_oipa.item():.4f}, Loss_on: {loss_on.item():.4f}')
    

# # Evaluation
# with torch.no_grad():
#     outputs = model(X_test)
#     predicted = (outputs >= 0.5).float()
#     accuracy = (predicted == y_test).float().mean()
#     print(f'Test Accuracy: {accuracy:.4f}')

# # Make predictions
# with torch.no_grad():
#     outputs = model(X_new)
#     predicted = (outputs >= 0.5).float()

c:\Users\alexs\anaconda3\envs\cs828\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch [1/50], Loss_sma90: 0.6291, Loss_oipa: 0.7264, Loss_on: 0.7136
Epoch [2/50], Loss_sma90: 0.7100, Loss_oipa: 0.6798, Loss_on: 0.6949
Epoch [3/50], Loss_sma90: 0.6477, Loss_oipa: 0.7826, Loss_on: 1.0638
Epoch [4/50], Loss_sma90: 0.6850, Loss_oipa: 0.7472, Loss_on: 0.5080
Epoch [5/50], Loss_sma90: 0.6946, Loss_oipa: 0.4010, Loss_on: 0.3976
Epoch [6/50], Loss_sma90: 0.7202, Loss_oipa: 0.6258, Loss_on: 0.2832
Epoch [7/50], Loss_sma90: 0.6682, Loss_oipa: 0.6652, Loss_on: 0.4722
Epoch [8/50], Loss_sma90: 0.6518, Loss_oipa: 0.2769, Loss_on: 0.4101
Epoch [9/50], Loss_sma90: 0.6561, Loss_oipa: 0.2015, Loss_on: 0.5378
Epoch [10/50], Loss_sma90: 0.6505, Loss_oipa: 0.7299, Loss_on: 0.8502
Epoch [11/50], Loss_sma90: 0.6486, Loss_oipa: 0.1625, Loss_on: 0.1232
Epoch [12/50], Loss_sma90: 0.7402, Loss_oipa: 0.6607, Loss_on: 0.1553
Epoch [13/50], Loss_sma90: 0.7147, Loss_oipa: 0.3114, Loss_on: 0.2934
Epoch [14/50], Loss_sma90: 0.7265, Loss_oipa: 0.1385, Loss_on: 0.0444
Epoch [15/50], Loss_sma90: 0.

In [15]:
# Single model eval
# is_correct = []
# for i, (X_train, y_train) in enumerate(validation_loader_sma90):
    

#         # print(X_train)
#         # print(y_train)
#         # Forward pass
#     outputs = model(X_train)
#     out = outputs.squeeze(1)
#     output_bin = (out >= 0.5).int().item()
#     correct = output_bin == int(y_train.item())
#     is_correct.append(correct)
# # Count ratio of trues and falses in is_correct
# true_count = sum(is_correct)
# false_count = len(is_correct) - true_count
# print(f"True count: {true_count}")
# print(f"False count: {false_count}")
# print(f"True ratio: {true_count / len(is_correct)}")


In [18]:
# Majority Voting Ensemble Eval
is_correct_confidence = []
is_correct_vote = []
is_correct_sma90 = []
is_correct_oipa = []
is_correct_on = []

for i, (X_train, y_train) in enumerate(validation_loader_sma90):
        # print(X_train)
        # print(y_train)
        # Forward pass
    X_train_sma90 = validation_loader_sma90.dataset.X_train[i].unsqueeze(0).float()
    X_train_oipa = validation_loader_oipa.dataset.X_train[i].unsqueeze(0).float()
    X_train_on = validation_loader_on.dataset.X_train[i].unsqueeze(0).float()
    Y_train_sma90 = validation_loader_sma90.dataset.Y_train[i]
    Y_train_oipa = validation_loader_oipa.dataset.Y_train[i]
    Y_train_on = validation_loader_on.dataset.Y_train[i]
    # Sanity check
    assert (Y_train_sma90.item() == Y_train_oipa.item())
    assert (Y_train_sma90.item() == Y_train_on.item())

    
    # print(outputs_sma90)
    outputs_sma90 = model_sma_90d(X_train_sma90).squeeze(1)
    outputs_oipa = model_open_inproportion_to_average(X_train_oipa).squeeze(1)
    outputs_on = model_open_normalized(X_train_on).squeeze(1)

    ## Ensemble confidence based on average of outputs
    confidence_avg_out = (outputs_sma90 + outputs_oipa + outputs_on) / 3
    confidence_output_bin = (confidence_avg_out >= 0.5).int().item()

    vote_sma90 = (outputs_sma90 >= 0.5).int().item()
    is_correct_sma90.append(vote_sma90 == int(y_train.item()))
    vote_oipa = (outputs_oipa >= 0.5).int().item()
    is_correct_oipa.append(vote_oipa == int(y_train.item()))
    vote_on = (outputs_on >= 0.5).int().item()
    is_correct_on.append(vote_on == int(y_train.item()))
    ## Ensemble voting based on majority voting
    vote = (vote_sma90 + vote_oipa + vote_on) >= 2
    correct_confidence = confidence_output_bin == int(y_train.item())
    correct_vote = vote == int(y_train.item())
    is_correct_confidence.append(correct_confidence)
    is_correct_vote.append(correct_vote)

    # out = outputs.squeeze(1)
    # output_bin = (out >= 0.5).int().item()
    # correct = output_bin == int(y_train.item())
    # is_correct.append(correct)
# Count ratio of trues and falses in is_correct
true_count_confidence = sum(is_correct_confidence)
false_count_confidence = len(is_correct_confidence) - true_count_confidence
print(f"True count (Confidence): {true_count_confidence}")
print(f"False count (Confidence): {false_count_confidence}")
print(f"True ratio (Confidence): {true_count_confidence / len(is_correct_confidence)}")
true_count_vote = sum(is_correct_vote)
false_count_vote = len(is_correct_vote) - true_count_vote
print(f"True count (Vote): {true_count_vote}")
print(f"False count (Vote): {false_count_vote}")
print(f"True ratio (Vote): {true_count_vote / len(is_correct_vote)}")
true_count_sma90 = sum(is_correct_sma90)
false_count_sma90 = len(is_correct_sma90) - true_count_sma90
print(f"True count (sma90): {true_count_sma90}")
print(f"False count (sma90): {false_count_sma90}")
print(f"True ratio (sma90): {true_count_sma90 / len(is_correct_sma90)}")
true_count_oipa = sum(is_correct_oipa)
false_count_oipa = len(is_correct_oipa) - true_count_oipa
print(f"True count (OIPA): {true_count_oipa}")
print(f"False count (OIPA): {false_count_oipa}")
print(f"True ratio (OIPA): {true_count_oipa / len(is_correct_oipa)}")
true_count_on = sum(is_correct_on)
false_count_on = len(is_correct_on) - true_count_on
print(f"True count (ON): {true_count_on}")
print(f"False count (ON): {false_count_on}")
print(f"True ratio (ON): {true_count_on / len(is_correct_on)}")


True count (Confidence): 91
False count (Confidence): 12
True ratio (Confidence): 0.883495145631068
True count (Vote): 85
False count (Vote): 18
True ratio (Vote): 0.8252427184466019
True count (sma90): 14
False count (sma90): 89
True ratio (sma90): 0.13592233009708737
True count (OIPA): 91
False count (OIPA): 12
True ratio (OIPA): 0.883495145631068
True count (ON): 92
False count (ON): 11
True ratio (ON): 0.8932038834951457


In [17]:
# # Count ratio of trues and falses in is_correct
# true_count = sum(is_correct)
# false_count = len(is_correct) - true_count
# print(f"True count: {true_count}")
# print(f"False count: {false_count}")
# print(f"True ratio: {true_count / len(is_correct)}")

In [20]:
# Save each model
directory = "classifier_models/"
if not os.path.exists(directory):
    os.makedirs(directory)
torch.save(model_sma_90d.state_dict(), directory +"sma_90d_model.pth")
torch.save(model_open_inproportion_to_average.state_dict(), directory +"open_inproportion_to_average_model.pth")
torch.save(model_open_normalized.state_dict(), directory +"open_normalized_model.pth")


In [ ]:
# Load in saved models
model_sma_90d = BinaryClassifier()
model_open_inproportion_to_average = BinaryClassifier()
model_open_normalized = BinaryClassifier()
model_sma_90d.load_state_dict(torch.load(directory +"sma_90d_model.pth"))
model_open_inproportion_to_average.load_state_dict(torch.load(directory +"open_inproportion_to_average_model.pth"))
model_open_normalized.load_state_dict(torch.load(directory +"open_normalized_model.pth"))


#